# Set Up 2023 Data

In [ ]:
import json
import pandas as pd

filepath = "Zillow-ChapelHill-Spring2023-18k-Properties.json"

# First, Load the JSON file and convery it to a pandas data frame
with open(filepath) as f:
  data2023 = json.load(f)
data2023.keys()


df_2023 = pd.DataFrame(data2023)

# Display the first few rows of the DataFrame
print(df_2023.head())


# Set up 2025 Data

In [ ]:

filepath0 = 'Zillow-March2025-dataset_part0.json'
filepath1 = 'Zillow-March2025-dataset_part1.json'
filepath2 = 'Zillow-March2025-dataset_part2.json'
filepath3 = 'Zillow-March2025-dataset_part3.json'

# Load the JSON files and convert them to pandas data frames
with open(filepath0) as f:
  data0 = json.load(f)
with open(filepath1) as f:
  data1 = json.load(f)
with open(filepath2) as f:
  data2 = json.load(f)
with open(filepath3) as f:
  data3 = json.load(f)
# Combine the data from all files into a single DataFrame
df0 = pd.DataFrame(data0)
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)
df_2025 = pd.concat([df0, df1, df2, df3], ignore_index=True)


In [ ]:
#  "address": {
#               "streetAddress": "2 Midstream Ct",
#               "city": "Chapel Hill",
#               "state": "NC",
#               "zipcode": "27517"
#             },
# find the 119 s graham st 
s_graham = df_2025[df_2025['address'].apply(lambda x: x['streetAddress'] == '119 S Graham St' and x['city'] == 'Chapel Hill' and x['state'] == 'NC')]
print(s_graham["daysOnZillow"])

# 2025 Walkable Locations

reduce data to locations that are <1 mile from center of campus 

In [ ]:
from geopy.distance import geodesic

# Define key campus locations (e.g., The Pit, Franklin Street)
unc_coordinates = (35.9106, -79.0472)

# Calculate walking/biking distance for each property
def calculate_distance_to_location(row, location_coords):
    property_coords = (row['latitude'], row['longitude'])
    return geodesic(property_coords, location_coords).miles

# Remove rows with missing or invalid latitude/longitude
df_2025 = df_2025.dropna(subset=['latitude', 'longitude'])

# Calculate distance from campus for each property
df_2025['distance_from_campus'] = df_2025.apply(
    lambda row: calculate_distance_to_location(row, unc_coordinates), axis=1
)

# Filter for properties within 5 miles of campus
df_2025_nearby = df_2025[df_2025['distance_from_campus'] <= 1]

# Display the filtered properties
print(df_2025_nearby[['streetAddress', 'city', 'state', 'price', 'bedrooms', 'bathrooms', 'distance_from_campus']])



# Property Type Filter

Focus on:
- Condos, townhomes, single family homes
- Units with 2–3 bedrooms (for possible roommates)





In [ ]:
# Clean up the 'homeType' column
df_2025_nearby.loc[:, 'homeType'] = df_2025_nearby['homeType'].str.lower()
df_2025 = df_2025_nearby[
    (df_2025_nearby['homeType'].isin(['condo', 'townhouse', 'single_family'])) &
    (df_2025_nearby['bedrooms'].between(2, 3))
]
# Display the filtered properties
print(df_2025[['streetAddress', 'city', 'homeType', 'price', 'bedrooms', 'bathrooms', 'distance_from_campus']])

# Time on Market / Turnover
- Look at average days on market for each property
- Higher turnover may signal student-friendly properties
- Low turnover could signal quiet, long-term-owner zones ideal for parents seeking stability

# Property Features
- In-unit laundry (major preference)
- Dedicated parking
- Safe, well-lit paths or sidewalks to campus
    

# Transit Routes

- Evaluate proximity to Chapel Hill Transit stops
- Bonus points for routes with direct connection to campus
